In [67]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
import base64
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps

#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from animal_shelter import AnimalShelter





###########################
# Data Manipulation / Model
###########################
# FIX ME change for your username and password and CRUD Python module name
username = "aacuser"
password = "2022snhu"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.read({}))



#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.A(href="http:://www.snhu.edu", children = html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()),style={'width': '100px', 'height':'100px'})),
    html.Center(html.B(html.H1('Animal Shelter'))),
    html.Hr(),
        
#FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.
    html.Div(className='row',
             style={'flex-wrao':'nowrap'},
             children=[dcc.Dropdown(id='dropdown-button', options=[
                 {"label": "Water Rescue", "value": "Water"},
                 {"label": "Mountain Rescue", "value": "Mountain"},
                 {"label": "Disaster Rescue", "value": "Disaster"},
                 {"label": "Reset", "value": "Reset"},
             ],
                                    value='Reset',
                                    clearable=False
                                    )
                  ]),
    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here 
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable="single",
        row_deletable=False,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current= 0,
        page_size= 10,
    ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ]),
    html.Tfoot("Animal Shelter Database by Nicholas Cleveland"),
    html.Tfoot("SNHU")
])

#############################################
# Interaction Between Components / Controller
#############################################

    
# Dropdown of animal types
@app.callback(
    Output('datatable-id', 'data'),
    [Input('dropdown-button', 'value')])
def on_click(radio_value):
    # start case
    if radio_value == "Reset":
        df = pd.DataFrame.from_records(shelter.read({}))
    elif radio_value == "Water":
        df = pd.DataFrame(list(shelter.read({"breed": {"$in": ["Labrador Retriever Mix", 
                                                               "Chesa Bay Retr",
                                                               "Newfoundland"]}, 
                                            "sex_upon_outcome":"Intact Female",
                                            "age_upon_outcome_in_weeks":{"$gte":26},
                                            "age_upon_outcome_in_weeks":{"$lte":156}})))
    elif radio_value == "Mountain":
        df = pd.DataFrame(list(shelter.read({"breed": {"$in": ["German Shepard", 
                                                               "Alaskan Malamute",
                                                               "Old English Sheepdog",
                                                               "Siberian Husky",
                                                               "Rottweiler"]},
                                             "sex_upon_outcome":"Intact Male",
                                             "age_upon_outcome_in_weeks":{"$gte":26},
                                             "age_upon_outcome_in_weeks":{"$lte":156}})))
    elif radio_value == "Disaster":
        df = pd.DataFrame(list(shelter.read({"breed": {"$in": ["Doberman Pinscher", 
                                                               "German Shepard",
                                                               "Golden Retriever",
                                                               "Bloodhound",
                                                               "Rottweiler"]},
                                             "sex_upon_outcome":"Intact Male",
                                             "age_upon_outcome_in_weeks":{"$gte":20},
                                             "age_upon_outcome_in_weeks":{"$lte":300}})))
    
   
    return df.to_dict('records')


@app.callback(
   Output('graph-id', "children"),
   [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    ##FIX ME ####
    #add code for chart of your choice (e.g. pie chart) #
    dff = pd.DataFrame.from_dict(view_data)
    breeds = []
    for i in range(0, dff.__len__() - 1):
        breeds.append(str(dff.iloc[i, 4]))
    return [
       dcc.Graph(            
           figure = {
           px.pie(dff, values=values, names=names, title='Percentage of breeds available')}
       )    
    ]

@app.callback(
   Output('map-id', "children"),
   [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
# FIXME: Add in the code for your geolocation chart
# If you completed the Module Six Assignment, you can copy in the code you created here.
    if view_data is None:
        return []
    else:
        # Add in the code for your geolocation chart
        dff = pd.DataFrame.from_dict(view_data)
        # Austin TX is at [30.75,-97.48]
        return [
            dl.Map(style={'width': '1000px', 'height': '500px'}, center=[dff.iloc[0, 13], dff.iloc[0, 14]], zoom=10,
                   children=[
                       dl.TileLayer(id="base-layer-id"),
                       # Marker with tool tip and popup
                       dl.Marker(position=[dff.iloc[0, 13], dff.iloc[0, 14]], children=[
                           dl.Tooltip(dff.iloc[0, 4]),
                           dl.Popup([
                               html.H1(dff.iloc[0, 9]),
                               html.P(dff.iloc[0, 8])
                           ])
                       ])
                   ])
        ]



app